In [1]:
import os
import copy
os.environ["CUDA_AVAILABLE_DEVICES"] = "0"
import numpy as np
import matplotlib.pyplot as plt
import torch
from lightly.loss import DINOLoss
from lightly.models.modules import DINOProjectionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.dino_transform import DINOTransform
from lightly.utils.scheduler import cosine_schedule
import torchvision
from torchvision.models import resnet18,efficientnet_b0
from torchvision import datasets, transforms
import torch.nn as nn
from copy import deepcopy
from torchvision import datasets, transforms
from astra.torch.metrics import f1_score,precision_score,recall_score
from astra.torch.utils import train_fn
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tqdm import tqdm,trange
import pandas as pd

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_data_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/data.pt"
test_data_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data_final/test_data.pt" 

In [ ]:
dataset = torchvision.datasets.ImageNet

In [5]:
train_data = torch.load(train_data_path)
test_data = torch.load(test_data_path)
train_images = train_data['images']
train_labels = train_data['labels']
test_images = test_data['images']
test_labels = test_data['labels']

In [6]:
train_labels.dtype, test_labels.dtype

(torch.uint8, torch.uint8)

In [7]:
train_images.shape,train_labels.shape

(torch.Size([25500, 3, 224, 224]), torch.Size([25500]))

In [8]:
test_images.shape, test_labels.shape

(torch.Size([4500, 3, 224, 224]), torch.Size([4500]))

In [9]:
data_transforms = transforms.Compose([
            transforms.Resize(224),
            transforms.RandomRotation(360),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
       
train_images = train_images.float()
test_images = test_images.float()
# transform data and convert into pytorch dataset fromat to load into dataloader
transformed_images = data_transforms(train_images)
transformed_images.shape

torch.Size([25500, 3, 224, 224])

In [10]:
train_labels.dtype, test_labels.dtype

(torch.uint8, torch.uint8)

In [3]:
dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14", pretrained=True)

Using cache found in /home/vannsh.jani/.cache/torch/hub/facebookresearch_dinov2_main
/home/vannsh.jani/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/vannsh.jani/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/vannsh.jani/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [11]:
dinov2_vits14

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-11): 12 x NestedTensorBlock(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (n

In [4]:
class DinoVisionTransformerClassifier(nn.Module):
    def __init__(self):
        super(DinoVisionTransformerClassifier, self).__init__()
        self.transformer = dinov2_vits14
        self.classifier = nn.Sequential(nn.Linear(384, 256), nn.ReLU(), nn.Linear(256, 2))

    def forward(self, x):
        x = self.transformer(x)
        x = self.transformer.norm(x)
        x = self.classifier(x)
        return x


model = DinoVisionTransformerClassifier()

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x1,x2,y1,y2 = train_test_split(train_images,train_labels,test_size=0.2,random_state=42,stratify=train_labels)
x1.shape,x2.shape,y1.shape,y2.shape

(torch.Size([20400, 3, 224, 224]),
 torch.Size([5100, 3, 224, 224]),
 torch.Size([20400]),
 torch.Size([5100]))

In [15]:
def train(model, X_train, y_train, criterion, epochs, lr, verbose, batch_size=32):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    model.to(device)
    epoch_losses = []
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    for epoch in trange(epochs, desc="Epochs"):
        batch_losses = []  # Track losses per batch

        for i in range(0, len(X_train), batch_size):
            optimizer.zero_grad()
            X_batch = X_train[i:i+batch_size].to(device)
            y_batch = y_train[i:i+batch_size].to(device)
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())

        # Calculate average loss for the epoch
        epoch_loss = sum(batch_losses) / len(batch_losses)
        epoch_losses.append(epoch_loss)
        
        if verbose:
            print(f"Epoch: {epoch} Loss: {epoch_loss}")

    return epoch_losses


In [8]:
total = 0
for p in list(model.parameters()):
    # print(p.size())
    total += p.numel()
total


22155650

In [16]:
epoch_losses = train(model,train_images,train_labels,nn.CrossEntropyLoss(),250,0.0001,True,256)

Epochs:   0%|          | 1/250 [01:21<5:37:28, 81.32s/it]

Epoch: 0 Loss: 0.25385498493909836


Epochs:   1%|          | 2/250 [02:37<5:24:26, 78.50s/it]

Epoch: 1 Loss: 0.2356398443877697


Epochs:   1%|          | 3/250 [03:53<5:18:08, 77.28s/it]

...

Epoch: 124 Loss: 0.00591265849499905


Epochs:  50%|█████     | 126/250 [2:40:18<3:03:42, 88.89s/it]

Epoch: 125 Loss: 0.005446548876770976


Epochs:  50%|█████     | 126/250 [2:41:39<2:39:05, 76.98s/it]


KeyboardInterrupt: 

In [ ]:
plt.plot(epoch_losses)

In [17]:
def get_metrics(y_pred,y_label):
    with torch.no_grad():
        f1 = f1_score(y_pred,y_label)
        precision = precision_score(y_pred,y_label)
        recall = recall_score(y_pred,y_label)
        return f1,precision,recall

def predict(model,X_test,y_test,batch_size = 32):
    result_dict = {}
    with torch.no_grad():
        model.eval()
        # incorporating batch size
        test_dataset = TensorDataset(X_test, y_test)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)
        y_pred = []
        for x, y in tqdm(test_loader):
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            y_pred.append(output)
        y_pred = torch.cat(y_pred)
        print(y_pred)
        y_pred = torch.argmax(y_pred,dim=1)
        print(y_pred)
        y_pred = y_pred.cpu()
        y_test = y_test.cpu()
        # print(y_pred.dtype,y_test.dtype)
        f1,precision,recall = get_metrics(y_pred,y_test)


        result_dict = {"f1":f1,"precision":precision,"recall":recall}
        return result_dict,y_pred,y_test


In [18]:
result,y_pred,y_test = predict(model,test_images,test_labels,batch_size=128)

100%|██████████| 36/36 [00:55<00:00,  1.54s/it]

tensor([[ 3.1660, -2.5737],
        [ 0.4591, -0.0945],
        [ 5.3940, -4.7773],
        ...,
        [ 0.2043,  0.5824],
        [ 0.3519,  0.3633],
        [ 0.8291, -0.1084]], device='cuda:0')
tensor([0, 0, 0,  ..., 1, 1, 0], device='cuda:0')


In [19]:
df = pd.DataFrame(result,index=[0])
df

,f1,precision,recall
0,tensor(0.0553),tensor(0.0532),tensor(0.0576)


In [ ]:
y_test[:100]

In [ ]:
for i in y_pred:
    if i.item() == 1:
        print(i)

In [ ]:
train_labels

In [ ]:
test_labels

In [1]:
%pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 757.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 1.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install lightly

Note: you may need to restart the kernel to use updated packages.


In [11]:
class DINO(torch.nn.Module):
    def __init__(self, backbone, input_dim):
        super().__init__()
        self.student_backbone = backbone
        self.student_head = DINOProjectionHead(
            input_dim, 512, 64, 2048, freeze_last_layer=1
        )
        self.teacher_backbone = copy.deepcopy(backbone)
        self.teacher_head = DINOProjectionHead(input_dim, 512, 64, 2048)
        deactivate_requires_grad(self.teacher_backbone)
        deactivate_requires_grad(self.teacher_head)

    def forward(self, x):
        y = self.student_backbone(x).flatten(start_dim=1)
        z = self.student_head(y)
        return z

    def forward_teacher(self, x):
        y = self.teacher_backbone(x).flatten(start_dim=1)
        z = self.teacher_head(y)
        return z

In [12]:
efficientnet_b0 = torchvision.models.efficientnet_b0()
backbone = nn.Sequential(*list(efficientnet_b0.children())[:-1])

In [13]:
backbone

Sequential(
  (0): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivation(
    

In [14]:
input_dim = 1280
model = DINO(backbone, input_dim).to(device)

/home/vannsh.jani/miniconda3/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [15]:
model

DINO(
  (student_backbone): Sequential(
    (0): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_activati

In [16]:
transform = DINOTransform()

In [17]:
train_dataset = TensorDataset(transformed_images, train_labels)
dataloader = DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True,
    drop_last=True,
    num_workers=8,
)

In [18]:
criterion = DINOLoss(
    output_dim=2048,
    warmup_teacher_temp_epochs=5,
)
criterion = criterion.to(device)

In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [20]:
epochs = 10

In [21]:
print("Starting Training")
for epoch in range(epochs):
    total_loss = 0
    momentum_val = cosine_schedule(epoch, epochs, 0.996, 1)
    for batch in tqdm(dataloader):
        views = batch[0]
        update_momentum(model.student_backbone, model.teacher_backbone, m=momentum_val)
        update_momentum(model.student_head, model.teacher_head, m=momentum_val)
        views = [view.to(device) for view in views]
        global_views = views[:2]
        teacher_out = [model.forward_teacher(view.unsqueeze(0)) for view in global_views]
        student_out = [model.forward(view.unsqueeze(0)) for view in views]
        loss = criterion(teacher_out, student_out, epoch=epoch)
        total_loss += loss.detach()
        loss.backward()
        # We only cancel gradients of student head.
        model.student_head.cancel_last_layer_gradients(current_epoch=epoch)
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / len(dataloader)
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")

Starting Training


100%|██████████| 99/99 [10:51<00:00,  6.58s/it] 


epoch: 00, loss: 8.04164


100%|██████████| 99/99 [10:17<00:00,  6.24s/it]


epoch: 01, loss: 7.62096


...

100%|██████████| 99/99 [10:29<00:00,  6.35s/it]


epoch: 03, loss: 7.62281


100%|██████████| 99/99 [10:08<00:00,  6.15s/it]


epoch: 04, loss: 7.62415


 16%|█▌        | 16/99 [01:42<08:26,  6.10s/it]

In [47]:
transformed_images.shape

torch.Size([25500, 3, 224, 224])

In [2]:
weights = torch.load("/home/vannsh.jani/brick_kilns/dinov2_vits14_pretrain.pth")

In [4]:
weights.keys()

dict_keys(['cls_token', 'pos_embed', 'mask_token', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.qkv.bias', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.ls1.gamma', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 'blocks.0.ls2.gamma', 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.qkv.bias', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.ls1.gamma', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias', 'blocks.1.ls2.gamma', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.attn.qkv.weight', 'blocks.2.attn.qkv.bias', 'blocks.2.attn.proj.weight', 'blocks.2.attn.proj.bias', 'blocks.2.ls1.gamma', 'blocks.2

In [5]:
weights["norm.weight"].shape,weights["norm.bias"].shape

(torch.Size([384]), torch.Size([384]))